In [1]:
import pandas as pd
import numpy as np

# 标称属性特征提取
- 报销剂型频次？
- 去的不同医院等级频次？
- 服务名称？

## 数据准备
提取辅表，以便获构建医院编号与类型的字典  
pd.read_hdf("data/df_train_time1.h5", key = "df_train_time1")

In [12]:
df_fee_detail = pd.read_hdf("data/df_fee_detail.h5", key = "df_fee_detail")

In [8]:
df_fee_detail.isnull().sum()/float(len(df_fee_detail))

顺序号           0.000000
医院编码          0.000000
三目统计项目        0.000000
三目服务项目名称      0.881171
三目医院服务项目名称    0.873052
剂型            0.000000
规格            0.000000
单价            0.000000
数量            0.000000
拒付原因编码        0.999988
拒付原因          0.999988
费用发生时间        0.000000
dtype: float64

In [13]:
category_cols = ['医院编码','三目服务项目名称','三目医院服务项目名称','剂型','规格']
df_fee_detail.drop(df_fee_detail.columns.difference(category_cols),axis=1,inplace=True)

In [19]:
df_fee_detail['剂型'].value_counts()/float(len(df_fee_detail))

                               5.087175e-01
片剂                             1.450977e-01
胶囊剂                            5.818334e-02
饮片                             5.242235e-02
注射剂                            4.610555e-02
其他                             3.696765e-02
0                              3.507890e-02
汤剂                             3.103277e-02
滴丸                             1.565706e-02
薄膜衣片                           9.502823e-03
肠溶片                            8.456733e-03
颗粒剂                            5.470617e-03
/                              3.880206e-03
注射液                            3.387928e-03
中药颗粒剂                          3.334813e-03
软胶囊                            3.114849e-03
乳剂                             2.749162e-03
丸剂                             2.158000e-03
中饮片颗粒剂                         1.875277e-03
糖衣片                            1.763688e-03
中草药                            1.647966e-03
素片                             1.340446e-03
冻干粉针                           1

In [20]:
df_fee_detail['三目服务项目名称'].value_counts()/float(len(df_fee_detail))

普通门诊挂号费---一级医院          4.711215e-02
糖尿病门诊诊察费（门特加收）          4.535703e-02
脑心通胶囊                   4.570098e-03
多导心电图检查自动分析             3.410430e-03
通心络胶囊                   2.781766e-03
肾炎康复片                   1.865940e-03
速效救心丸                   1.814967e-03
依诺肝素钠注射液                1.432289e-03
低分子量肝素钙注射液              1.196559e-03
普通门诊挂号费---二级医院          1.090940e-03
肝素钠注射液                  9.421542e-04
肺力咳合剂                   9.277655e-04
牛黄清心丸                   7.082612e-04
低分子肝素钠注射液               6.294294e-04
低分子量肝素钠注射液              5.536591e-04
注射用低分子量肝素钠              3.266543e-04
地奥心血康软胶囊                3.168577e-04
丹芪偏瘫胶囊                  2.755284e-04
多导心电图                   2.680279e-04
达肝素钠注射液                 1.810834e-04
那屈肝素钙注射液                1.767974e-04
病床紫外线消毒                 1.717461e-04
清肺消炎丸                   1.673070e-04
专家门诊挂号费---主任医师          1.665416e-04
癌胚抗原（电化学发光法）            1.653171e-04
稳心颗粒                    1.549082e-04
血栓心脉宁片                  1.388357e-04
专

In [21]:
df_fee_detail['三目医院服务项目名称'].value_counts()/float(len(df_fee_detail))

普通门诊挂号费---一级医院         4.494757e-02
糖尿病门诊诊察费（门特加收）         2.055090e-02
糖尿病门诊诊察费               1.507493e-02
挂号                     5.103093e-03
普通门诊挂号费                3.611872e-03
脑心通胶囊                  3.479465e-03
三级医院-普通门诊挂号费           2.193359e-03
通心络胶囊                  2.064626e-03
糖尿病门诊诊察费加收             1.810987e-03
门诊诊查费糖尿病加收             1.787261e-03
肾炎康复片                  1.490915e-03
多导心电图检查自动分析            1.424635e-03
速效救心丸                  1.377948e-03
糖尿病门诊诊查费（门特加收）         1.252583e-03
依诺肝素钠注射液               1.109308e-03
普通门诊挂号费---二级医院         8.607202e-04
多导心电图检查自动分析*           8.056145e-04
糖尿病加收                  7.857153e-04
肺力咳合剂                  7.794393e-04
肝素钠注射液                 7.740818e-04
通心络胶囊*                 6.629520e-04
多导心电图                  5.120237e-04
低分子肝素钠注射液[意大利]<*5%>    4.915121e-04
低分子量肝素钙注射液             4.096189e-04
挂号诊查费                  3.990570e-04
牛黄清心丸                  3.880359e-04
低分子量肝素钠注射液             3.805354e-04
糖尿病门诊诊查费加收             3.672

## 医院类型字典

In [30]:
df_fee_detail['三目医院服务项目名称'] = df_fee_detail['三目医院服务项目名称'].fillna('')

In [36]:
df_fee_detail['三目医院服务项目名称'][df_fee_detail['三目医院服务项目名称'].str.find('级') != -1].str.split('-')

97         [普通门诊挂号费, , , 一级医院]
99         [普通门诊挂号费, , , 一级医院]
120        [普通门诊挂号费, , , 一级医院]
122        [普通门诊挂号费, , , 一级医院]
124        [普通门诊挂号费, , , 一级医院]
161        [普通门诊挂号费, , , 一级医院]
206        [普通门诊挂号费, , , 一级医院]
215        [普通门诊挂号费, , , 一级医院]
275        [普通门诊挂号费, , , 一级医院]
302        [普通门诊挂号费, , , 一级医院]
319        [普通门诊挂号费, , , 一级医院]
340        [普通门诊挂号费, , , 一级医院]
409        [普通门诊挂号费, , , 一级医院]
421        [普通门诊挂号费, , , 一级医院]
422        [普通门诊挂号费, , , 一级医院]
431        [普通门诊挂号费, , , 一级医院]
433        [普通门诊挂号费, , , 一级医院]
440        [普通门诊挂号费, , , 一级医院]
460        [普通门诊挂号费, , , 一级医院]
465        [普通门诊挂号费, , , 一级医院]
530        [普通门诊挂号费, , , 一级医院]
535        [普通门诊挂号费, , , 一级医院]
611        [普通门诊挂号费, , , 一级医院]
614        [普通门诊挂号费, , , 一级医院]
733        [普通门诊挂号费, , , 二级医院]
773        [普通门诊挂号费, , , 一级医院]
831        [普通门诊挂号费, , , 一级医院]
835            [三级医院, 普通门诊挂号费]
840        [普通门诊挂号费, , , 二级医院]
842        [普通门诊挂号费, , , 一级医院]
                  ...         
6532052        [三级医院, 普通门诊挂号费]
6532087 

In [47]:
l = ['六级医院', '普通门诊挂号费','']
l.sort()
print l[1]

六级医院


In [52]:
df_hospital_type = df_fee_detail[['医院编码','三目医院服务项目名称']][df_fee_detail['三目医院服务项目名称'].str.find('医院')!= -1]

In [65]:
def type_hospital1(x):
    x = sorted(x.split('-'))
    try:
        return x[-2]
    except:
        return x[-1]
def type_hospital2(x):
    x = sorted(x.split('—'))
    try:
        return x[-2]
    except:
        return x[-1]
def type_hospital3(x):
    x = sorted(x.split('┄'))
    try:
        return x[-2]
    except:
        return x[-1]

In [66]:
df_hospital_type['三目医院服务项目名称'] = df_hospital_type['三目医院服务项目名称'].apply(type_hospital1).apply(type_hospital2).apply(type_hospital3).value_counts()

一级医院              294332
三级医院               14345
二级医院                5623
（偏瘫）床位费一级医院观察床        16
Name: 三目医院服务项目名称, dtype: int64

In [67]:
df_hospital_type['三目医院服务项目名称'] = df_hospital_type['三目医院服务项目名称'].apply(type_hospital1).apply(type_hospital2).apply(type_hospital3)

In [72]:
df_hospital_type[df_hospital_type['三目医院服务项目名称'] =='（偏瘫）床位费一级医院观察床'] = '一级医院'

In [73]:
df_hospital_type['三目医院服务项目名称'].value_counts()

一级医院    294348
三级医院     14345
二级医院      5623
Name: 三目医院服务项目名称, dtype: int64

In [78]:
df_hospital_type = df_hospital_type.drop_duplicates('医院编码').sort_values('医院编码').rename(columns={'三目医院服务项目名称':'医院类型'})

In [97]:
df_hospital_type['医院类型'].value_counts()

一级医院    346
二级医院     15
三级医院      1
Name: 医院类型, dtype: int64

In [83]:
print '\xe4\xb8\x80\xe7\xba\xa7\xe5\x8c\xbb\xe9\x99\xa2'

一级医院


In [85]:
hospital_type_map = dict(list(df_hospital_type.groupby('医院类型')['医院编码']))

In [90]:
import pickle
pickle.dump(hospital_type_map,open('data/hospital_type_map.pkl','w'))

In [93]:
hospital_type_map = pickle.load(open('data/hospital_type_map.pkl','r'))

In [94]:
hospital_type_map

{'\xe4\xb8\x80\xe7\xba\xa7\xe5\x8c\xbb\xe9\x99\xa2': 453678        2
 488586        3
 1023          5
 3495          7
 11735         8
 14055         9
 17588        10
 7681         11
 23366        14
 4822         16
 3671         18
 12272        19
 12206        20
 811156       24
 5182         25
 6330         26
 14768        27
 9285         28
 2213         29
 2899         30
 10264        31
 122          32
 14919        34
 527142       35
 629648       36
 452424       37
 7385         38
 467480       41
 489530       42
 422631       43
            ... 
 4759323    1368
 351886     1386
 138056     1393
 436307     1403
 388459     1412
 1973160    1415
 73478      1416
 6078       1426
 215        1431
 6434712    1437
 43377      1441
 548279     1442
 29188      1444
 389043     1447
 64898      1451
 1382       1468
 1559704    1485
 449627     1488
 377675     1490
 458400     1507
 15917      1513
 2400814    1536
 1455501    1537
 357242     1558
 2779329    1